<a href="https://colab.research.google.com/github/zhangou888/NN/blob/main/GNN_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Node classification on the Cora citation network using GCN (Graph Convolutional Network)

## Step 1: Install Required Libraries

In [1]:
# Install PyTorch (use CPU version; change link if using GPU)
!pip install torch torchvision torchaudio
# Install PyTorch Geometric dependencies
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.2.0+cpu.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.2.0+cpu.html
!pip install torch-geometric

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 33.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

## Step 2: Import Libraries

In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv

/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_sparse/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


## Step 3: Load the Cora Dataset

- data.x: Node features (shape: [num_nodes, num_node_features])

- data.edge_index: Graph connectivity (COO format: [2, num_edges])

- data.y: Node labels (shape: [num_nodes])

- data.train_mask, data.val_mask, data.test_mask: Boolean masks

In [3]:
# Load the Cora dataset from the Planetoid collection
dataset = Planetoid(root='data/Planetoid', name='Cora')

# Access the graph object (Cora has only one graph)
data = dataset[0]

Processing...
Done!


## Step 4: Define the GCN Model



In [4]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # First GCN layer: from input features to 16 hidden units
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        # Second GCN layer: from 16 hidden units to number of classes
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        # First GCN layer + ReLU activation
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # Dropout for regularization (only during training)
        x = F.dropout(x, training=self.training)
        # Second GCN layer
        x = self.conv2(x, edge_index)
        # Log Softmax for classification
        return F.log_softmax(x, dim=1)

## Step 5: Initialize Model, Optimizer, and Move to Device

In [5]:
# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GCN().to(device)
data = data.to(device)

# Adam optimizer with learning rate and L2 regularization
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

## Step 6: Define Training Function

In [ ]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data)
    # Compute loss only on training nodes
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

## Step 7: Define Test Function

In [ ]:
def test():
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)  # Get class predictions
    # Calculate accuracy on test set
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    return acc

##  Step 8: Run Training Loop
## Notes
- **Model**: 2-layer GCN

- **Task**: Semi-supervised node classification

- **Dataset**: Cora — 2,708 nodes, 1,433 features, 7 classes

- **Masking**: Only part of the nodes are used for training (train_mask), the rest for testing

In [ ]:
for epoch in range(1, 201):  # Train for 200 epochs
    loss = train()
    if epoch % 10 == 0:
        acc = test()
        print(f'Epoch {epoch:03d}, Loss: {loss:.4f}, Test Acc: {acc:.4f}')

## Step 9: Save the Model to File

In [ ]:
# Save the trained model to a file
torch.save(model.state_dict(), 'gcn_model.pth')
print("Model saved to gcn_model.pth")

# You can download the file from Colab if needed with:
from google.colab import files
files.download('gcn_model.pth')

## Step 10: Load the Model from File

In [ ]:
# Recreate the model structure and load the saved weights
model = GCN().to(device)
model.load_state_dict(torch.load('gcn_model.pth'))
model.eval()  # Set to evaluation mode
print("Model loaded successfully.")

## Step 11: Make Predictions


In [ ]:
# Forward pass to get predictions
out = model(data)
pred = out.argmax(dim=1)  # Predicted class for all nodes

# Example: Get predicted label for node 100
node_id = 100
predicted_class = pred[node_id].item()
true_class = data.y[node_id].item()

print(f"Node {node_id} — Predicted Class: {predicted_class}, True Class: {true_class}")